# ALL_RSS

## Given that RSS FEEDs update frequently we need to find a way to include the new jobs into the current DB without losing the "old" jobs.

### In brief...

### For every iteration, the df is inspected and then filtered by its pubDate (if there's one). Finally, it creates a new df that will be transformed into a new table in the local postgreSQL DB.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import mysql.connector
import psycopg2
import numpy as np

In [2]:
df = pd.read_csv('/Users/juanreyesgarcia/Library/CloudStorage/OneDrive-FundacionUniversidaddelasAmericasPuebla/DEVELOPER/PROJECTS/CRAWLER_ALL/yummy_soup_rss.csv')

##### Play with the settings...

In [3]:
pd.set_option('display.max_colwidth', 150)

In [4]:
pd.set_option("display.max_rows", None)

# EDA Checklist 

In [5]:
df.head()

,title,link,description,pubDate,location
0,Senior Clojure Developer Codete,https://ClojureJobboard.com/clojure-job/ib-remote-senior-clojure-developer-krakow-codete-remotework.html,Remote Codete is not just a software company its a place where techenthusiasts can grow by doing what they love and feel valued for what they are...,NaN,NaN
1,Senior Software Engineer CIS Cloud Services Centripetal,https://ClojureJobboard.com/clojure-job/ha-remote-senior-software-engineer-cis-cloud-services-centripetal-remotework.html,Remote Do you want to be at the forefront of intelligencedriven cybersecurity We at Centripetal are innovators of disruptive cybersecurity soluti...,NaN,NaN
2,FullStack Clojure Developer Appear,https://ClojureJobboard.com/clojure-job/ia-remote-europe-full-stack-clojure-developer-appear-remotework.html,Remote At Appear were looking for a FullStack Clojure Developer to join our team Fulltime Oslo OR Remote About the role Were looking for a Cloju...,NaN,NaN
3,Software Engineer Clojure Backend Comcast,https://ClojureJobboard.com/clojure-job/ic-software-engineer-clojure-backend-philadelphia-comcast.html,Philadelphia United States Comcasts Technology Product Experience organization works at the intersection of media and technology Our innovative ...,NaN,NaN
4,Senior Application Developer Clojure Pennymac,https://ClojureJobboard.com/clojure-job/ii-senior-application-developer-clojure-agoura-hills-pennymac.html,Agoura Hills United States At Pennymac our people are the foundation of our success and at the heart of our dynamic work culture Together we work...,NaN,NaN


In [6]:
df.tail()

,title,link,description,pubDate,location
1390,WordPress Designer,https://jobs.wordpress.net/job/wordpress-designer-54/,ob Title WordPress Designer for httpjomirecordonline Job Description We are seeking a skilled and experienced WordPress Designer to join our team ...,Sun 05 Mar 2023 141018 0000,NaN
1391,WordPress Developer,https://jobs.wordpress.net/job/wordpress-developer-793/,Job Title WordPress Developer for httpjomerecordonline Job Description We are seeking a talented and experienced WordPress Developer to join our t...,Sun 05 Mar 2023 141009 0000,NaN
1392,Safety Developer WordPress Maintenace Cause Marketing Partner,https://jobs.wordpress.net/job/safety-developer-wordpress-maintenace-cause-marketing-partner/,ASF an NPO leader in driving safety since 1988 has international lifesaving research and valuable IP at carsafeorg This public safety work require...,Sun 05 Mar 2023 140950 0000,NaN
1393,Engineering Manager Remote,https://jobs.wordpress.net/job/engineering-manager-remote-6/,Are you an inspirational forwardthinking and peoplecentric seasoned senior engineering manager Do you excel at transforming vision and strategy in...,Fri 03 Mar 2023 131837 0000,NaN
1394,Copywriter FrenchEnglish Remote,https://jobs.wordpress.net/job/copywriter-french-english-remote-2/,Are you passionate about triggering a real emotion for your reader Are you able to convey complex ideas using as few words as possible Do you love...,Fri 03 Mar 2023 131748 0000,NaN


### Describe the DF (current)

In [7]:
df.describe()

,title,link,description,pubDate,location
count,1395,1395,1154,1218,650
unique,1271,1395,1120,1158,182
top,DevOps Engineer,https://ClojureJobboard.com/clojure-job/ib-remote-senior-clojure-developer-krakow-codete-remotework.html,United States,Mon 06 Mar 2023 080000 0200,Remote
freq,7,1,11,7,88


### Checking for missing values DF (current)

In [8]:
df.isnull().sum()

title            0
link             0
description    241
pubDate        177
location       745
dtype: int64

### Fill missing values with "NaN" DF (current)

In [9]:
# Fill missing values with "NaN"
df.fillna("NaN", inplace=True)

In [10]:
df.isnull().sum()

title          0
link           0
description    0
pubDate        0
location       0
dtype: int64

### Checking for duplicates DF (current)

In [11]:
df.duplicated().sum()

0

In [12]:
df.describe()

,title,link,description,pubDate,location
count,1395,1395,1395,1395,1395
unique,1271,1395,1121,1159,183
top,DevOps Engineer,https://ClojureJobboard.com/clojure-job/ib-remote-senior-clojure-developer-krakow-codete-remotework.html,NaN,NaN,NaN
freq,7,1,241,177,745


In [13]:
df.dtypes

title          object
link           object
description    object
pubDate        object
location       object
dtype: object

# From string to date time...

In [14]:
df.loc[:, 'pubDate']  = pd.to_datetime(df['pubDate'], errors="coerce", format="%a %d %b %Y", exact=False)

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_13165/1359017417.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'pubDate']  = pd.to_datetime(df['pubDate'], errors="coerce", format="%a %d %b %Y", exact=False)


## Filter rows by a date range (this reduces the number of rows... duh)

In [15]:
# filter rows by a date range
start_date = pd.to_datetime('2016-01-01')
end_date = pd.to_datetime('2023-02-15')

#for df
date_range_filter = (df['pubDate'] >= start_date) & (df['pubDate'] <= end_date)
df = df.loc[~date_range_filter]

In [16]:
df = df.sort_values(by='pubDate')

In [17]:
df.describe(datetime_is_numeric=True)

,pubDate
count,409
mean,2023-02-27 04:48:42.249388800
min,2023-02-16 00:00:00
25%,2023-02-23 00:00:00
50%,2023-03-01 00:00:00
75%,2023-03-03 00:00:00
max,2023-03-06 00:00:00


In [18]:
df

,title,link,description,pubDate,location
806,Johns Hopkins University is hiring for a position of Web Designer in Remote,https://dribbble.com/jobs/203090-Web-Designer?source=feed,NaN,2023-02-16,NaN
1354,Aha Senior Product Marketing Manager,https://weworkremotely.com/remote-jobs/aha-senior-product-marketing-manager,Headquarters United States URL httpwwwahaio Aha helps companies build lovable software We provide the worlds 1 product development tools Aha Ro...,2023-02-16,NaN
63,Senior Full Stack Developer NET at AAOS Rosemont IL USA,https://www.crunchboard.com/jobs/197057956-senior-full-stack-developer-net-at-aaos,You are an enthusiastic thought leader who coaches and mentors those around you You are highly creative and understand how to balance creativity and,2023-02-16,NaN
64,Principal software engineer SQL API at Cube Dev San Francisco CA USA,https://www.crunchboard.com/jobs/197021714-principal-software-engineer-sql-api-at-cube-dev,At Cube Dev we are building a technology stack for modern analytics If you are fascinated by the software that powers large technology companies but,2023-02-16,NaN
889,Drupal FrontEnd Engineer Remote US amp CANADA position is open kanopistudios,https://jobs.drupal.org/kanopi-studios/job/17548,United States,2023-02-16,NaN
807,Voormedia is hiring for a position of Product Designer in Amsterdam Netherlands,https://dribbble.com/jobs/203058-Product-Designer?source=feed,NaN,2023-02-16,NaN
802,Riva Health is hiring for a position of Lead Product Designer Riva Health in United States Canada,https://dribbble.com/jobs/203123-Lead-Product-Designer-Riva-Health?source=feed,NaN,2023-02-16,NaN
1260,Full Stack Developer,https://vuejobs.com/jobs/my-worksheet-maker-full-stack-developer?utm_source=rssfeed,Employer My Worksheet Maker Location Full Stack Developer FullTime RemotenbspAre you a skilled developer looking to use your technical expertise ...,2023-02-16,NaN
1259,UX and Design Focused Senior Frontend Developer US Only,https://vuejobs.com/jobs/drive-commerce-ux-and-design-focused-senior-front-end-developer?utm_source=rssfeed,Employer Drive Commerce Location United States We are building the nextgeneration ecommerce personalization platform and we are looking for you if...,2023-02-16,NaN
360,Software Engineer Frontend,https://cryptojobslist.com/jobs/software-engineer-frontend-coinbase-remote-india,Tags Cryptocurrency Jobs Blockchain Remote Jobs Web3 Non Tech Jobs Cryptocurrency Trading Jobs Web3 Legal Jobs Blockchain Security Jobs Cry...,2023-02-17,Remote India


In [19]:
df.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_13165/3627053830.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe()


,title,link,description,pubDate,location
count,647,647,647,409,647
unique,583,647,510,19,73
top,Senior Software Engineer,https://dribbble.com/jobs/203090-Web-Designer?source=feed,NaN,2023-03-06 00:00:00,NaN
freq,4,1,115,72,482
first,NaN,NaN,NaN,2023-02-16 00:00:00,NaN
last,NaN,NaN,NaN,2023-03-06 00:00:00,NaN


In [20]:
df.dtypes

title                  object
link                   object
description            object
pubDate        datetime64[ns]
location               object
dtype: object

In [21]:
df['title'] = df['title'].astype(str)
df['link'] = df['link'].astype(str)
df['description'] = df['description'].astype(str)
df['location'] = df['location'].astype(str)

In [22]:
df.dtypes

title                  object
link                   object
description            object
pubDate        datetime64[ns]
location               object
dtype: object

###### Remember 'object' is the pandas data type that is used to represent string data.

In [23]:
df1 = df.copy()

In [24]:
df['description'] = df['description'].str.slice(0, 1000)

In [25]:
# replace NaT values in the DataFrame with None
df = df.replace({np.nan: None, pd.NaT: None})

In [26]:
df

,title,link,description,pubDate,location
806,Johns Hopkins University is hiring for a position of Web Designer in Remote,https://dribbble.com/jobs/203090-Web-Designer?source=feed,NaN,2023-02-16 00:00:00,NaN
1354,Aha Senior Product Marketing Manager,https://weworkremotely.com/remote-jobs/aha-senior-product-marketing-manager,Headquarters United States URL httpwwwahaio Aha helps companies build lovable software We provide the worlds 1 product development tools Aha Ro...,2023-02-16 00:00:00,NaN
63,Senior Full Stack Developer NET at AAOS Rosemont IL USA,https://www.crunchboard.com/jobs/197057956-senior-full-stack-developer-net-at-aaos,You are an enthusiastic thought leader who coaches and mentors those around you You are highly creative and understand how to balance creativity and,2023-02-16 00:00:00,NaN
64,Principal software engineer SQL API at Cube Dev San Francisco CA USA,https://www.crunchboard.com/jobs/197021714-principal-software-engineer-sql-api-at-cube-dev,At Cube Dev we are building a technology stack for modern analytics If you are fascinated by the software that powers large technology companies but,2023-02-16 00:00:00,NaN
889,Drupal FrontEnd Engineer Remote US amp CANADA position is open kanopistudios,https://jobs.drupal.org/kanopi-studios/job/17548,United States,2023-02-16 00:00:00,NaN
807,Voormedia is hiring for a position of Product Designer in Amsterdam Netherlands,https://dribbble.com/jobs/203058-Product-Designer?source=feed,NaN,2023-02-16 00:00:00,NaN
802,Riva Health is hiring for a position of Lead Product Designer Riva Health in United States Canada,https://dribbble.com/jobs/203123-Lead-Product-Designer-Riva-Health?source=feed,NaN,2023-02-16 00:00:00,NaN
1260,Full Stack Developer,https://vuejobs.com/jobs/my-worksheet-maker-full-stack-developer?utm_source=rssfeed,Employer My Worksheet Maker Location Full Stack Developer FullTime RemotenbspAre you a skilled developer looking to use your technical expertise ...,2023-02-16 00:00:00,NaN
1259,UX and Design Focused Senior Frontend Developer US Only,https://vuejobs.com/jobs/drive-commerce-ux-and-design-focused-senior-front-end-developer?utm_source=rssfeed,Employer Drive Commerce Location United States We are building the nextgeneration ecommerce personalization platform and we are looking for you if...,2023-02-16 00:00:00,NaN
360,Software Engineer Frontend,https://cryptojobslist.com/jobs/software-engineer-frontend-coinbase-remote-india,Tags Cryptocurrency Jobs Blockchain Remote Jobs Web3 Non Tech Jobs Cryptocurrency Trading Jobs Web3 Legal Jobs Blockchain Security Jobs Cry...,2023-02-17 00:00:00,Remote India


# PostgreSQL

### This code creates a new table per iteration

In [27]:
# create a connection to the PostgreSQL database
cnx = psycopg2.connect(user='postgres', password='3312', host='localhost', port='5432', database='postgres')

# create a cursor object
cursor = cnx.cursor()

# get the name of the next table to create
get_table_name_query = '''
    SELECT COUNT(*) FROM information_schema.tables
    WHERE table_name LIKE 'mytable_%'
'''
cursor.execute(get_table_name_query)
result = cursor.fetchone()
next_table_number = result[0] + 1
next_table_name = 'mytable_{}'.format(next_table_number)

# prepare the SQL query to create a new table
create_table_query = '''
    CREATE TABLE {} (
        title VARCHAR(255),
        link VARCHAR(255),
        description VARCHAR(1000),
        pubDate TIMESTAMP,
        location VARCHAR(255)
    )
'''.format(next_table_name)

# execute the create table query
cursor.execute(create_table_query)

# insert the DataFrame into the PostgreSQL database as a new table
for index, row in df.iterrows():
    insert_query = '''
        INSERT INTO {} (title, link, description, pubDate, location)
        VALUES (%s, %s, %s, %s, %s)
    '''.format(next_table_name)
    values = (row['title'], row['link'], row['description'], row['pubDate'], row['location'])
    cursor.execute(insert_query, values)

# commit the changes to the database
cnx.commit()

# close the cursor and connection
cursor.close()
cnx.close()

# MERGE 

In [28]:
#merged = pd.merge(df, df_a, on=['title','link','description','pubDate','location'], how='outer')

In [29]:
#merged.describe()

#### Check for duplicatesss...

In [30]:
#merged.duplicated(subset='link').sum()

#### Drop them...

In [31]:
#df1 = merged.drop_duplicates(subset=['link'])

In [32]:
#df1.duplicated(subset='link').sum()

In [33]:
#df1.describe()

# Save a copy of the df back into yummy_soup_rss_copy.csv so in the next iterations you will merge the output of the crawler with this df (basically making a loop)

## By doing this we will be able to merge the df that comes from the crawler and the merged df. If this was not done (& we just call the yummy_soup_rss_copy.csv file we would only merge those rows with the new ones, without including the data that was already merged)

In [34]:
#df1_copy = df1.copy()
#df1_copy.to_csv('yummy_soup_rss_copy.csv', index=False)

### Explore categorical data

For categorical data, check the number of unique values in each column 

In [35]:
#df1.nunique()

In [36]:
# Visualise the distribution of a categorical column using a bar chart
#df1['pubDate'].value_counts().plot(kind='bar', figsize=(14, 8))
#plt.show()

### Sort the dataframe by the date column in ascending order

In [37]:
#df1 = df1.sort_values(by='pubDate', ascending=False)

### Reset index...

In [38]:
#df1.reset_index(drop=True)

# GIVE ME THOSE JOBSSSS

## Only WorldWide - 2023

#### Given that some dates are NaN it is impossible to create a perfectly tailored df of solely worldwide jobs

##### I have tried to narrow down the search but there is still some jobs that are not fully worldwide. Although, the majority is indeed from anywhere in the world.

In [39]:
key_words = ['worldwide', 'international', 'anywhere']
regex = '|'.join(key_words)


worldwide = df1[df1['title'].str.contains(regex, case=False) | df1['description'].str.contains(regex, case=False) | df1['location'].str.contains(regex, case=False)]

In [40]:
worldwide.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_13165/3825536958.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  worldwide.describe()


,title,link,description,pubDate,location
count,134,134,134,75,134
unique,131,134,132,18,22
top,Senior Software Engineer,https://weworkremotely.com/remote-jobs/aha-senior-product-marketing-manager,Remote Gcore in the nutshell We are an international leader in public cloud and edge computing content delivery hosting and security solutions We...,2023-03-06 00:00:00,NaN
freq,2,1,2,15,66
first,NaN,NaN,NaN,2023-02-16 00:00:00,NaN
last,NaN,NaN,NaN,2023-03-06 00:00:00,NaN


In [41]:
worldwide

,title,link,description,pubDate,location
1354,Aha Senior Product Marketing Manager,https://weworkremotely.com/remote-jobs/aha-senior-product-marketing-manager,Headquarters United States URL httpwwwahaio Aha helps companies build lovable software We provide the worlds 1 product development tools Aha Ro...,2023-02-16,NaN
362,Senior Software Engineer,https://cryptojobslist.com/jobs/senior-software-engineer-coinbase-remote-india,Tags Cryptocurrency Jobs Web3 Remote Jobs Blockchain Non Tech Jobs Web3 Trading Jobs Cryptocurrency Legal Jobs Blockchain Developer Jobs We...,2023-02-17,Remote India
358,Software Engineer,https://cryptojobslist.com/jobs/software-engineer-coinbase-remote-india,Tags Cryptocurrency Jobs Cryptocurrency Remote Jobs Web3 Non Tech Jobs Blockchain Trading Jobs Web3 Legal Jobs Blockchain Security Jobs Blo...,2023-02-17,Remote India
357,Accounts Payable Analyst,https://cryptojobslist.com/jobs/accounts-payable-analyst-parity-technologies-london-england-united-kingdom,Tags Cryptocurrency Jobs Web3 Analyst Jobs Web3 Operations Jobs Web3 Finance Jobs Blockchain Accounting Jobs Blockchain Full Time Jobs Accou...,2023-02-17,London England United Kingdom
356,Sr DevOps Engineer New Zealand Fiji GMT12,https://cryptojobslist.com/jobs/sr-devops-engineer-new-zealand-fiji-gmt-12-polygon-auckland,Tags Cryptocurrency Jobs Cryptocurrency Remote Jobs Blockchain Devops Jobs Web3 Security Jobs Blockchain Developer Jobs Web3 Engineering Job...,2023-02-17,Auckland
347,Regulatory Strategy Counsel Europe,https://cryptojobslist.com/jobs/regulatory-strategy-counsel-europe-kraken-digital-asset-exchange-amsterdam,Tags Cryptocurrency Jobs Blockchain Remote Jobs Blockchain Non Tech Jobs Blockchain Legal Jobs Cryptocurrency Exchange Jobs Cryptocurrency C...,2023-02-17,Amsterdam
352,UIUX Design Intern UED,https://cryptojobslist.com/jobs/ui-ux-design-intern-ued-binance-asia,Tags Cryptocurrency Jobs Web3 Trading Jobs Blockchain Internship Jobs Web3 Engineering Jobs Web3 Ui Ux Jobs Web3 Exchange Jobs Cryptocurren...,2023-02-17,Asia
353,Junior AffiliateInfluencer Marketing Binance Accelerator Programme 6 months,https://cryptojobslist.com/jobs/junior-affiliate-influencer-marketing-binance-accelerator-programme-6-months-binance-global,Tags Cryptocurrency Jobs Blockchain Marketing Jobs Blockchain Trading Jobs Web3 Exchange Jobs Web3 Web3 Jobs Web3 Junior Level Jobs Blockch...,2023-02-17,Global
328,HR Payroll Manager FTCMaternity Cover,https://cryptojobslist.com/jobs/hr-payroll-manager-ftc-maternity-cover-gsr-mayfair-greater-london,Tags Cryptocurrency Jobs Cryptocurrency Trading Jobs Cryptocurrency Finance Jobs Web3 Manager Jobs About usFounded in 2013 GSR is a crypto mar...,2023-02-18,Mayfair Greater London
1352,YAGRO LTD Senior DevOps Engineer,https://weworkremotely.com/remote-jobs/yagro-ltd-senior-devops-engineer,Headquarters Cambridge URL httpswwwyagrocom The agricultural industry is undergoing a profound change Tightening regulatory requirements reduc...,2023-02-20,NaN


## Analyst - WorldWide

In [42]:

analyst = worldwide[worldwide['title'].str.contains('analyst', case=False) | worldwide['description'].str.contains('analyst', case=False)]


In [43]:
analyst.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_13165/1449045089.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  analyst.describe()


,title,link,description,pubDate,location
count,4,4,4,2,4
unique,4,4,4,2,4
top,Accounts Payable Analyst,https://cryptojobslist.com/jobs/accounts-payable-analyst-parity-technologies-london-england-united-kingdom,Tags Cryptocurrency Jobs Web3 Analyst Jobs Web3 Operations Jobs Web3 Finance Jobs Blockchain Accounting Jobs Blockchain Full Time Jobs Accou...,2023-02-17 00:00:00,London England United Kingdom
freq,1,1,1,1,1
first,NaN,NaN,NaN,2023-02-17 00:00:00,NaN
last,NaN,NaN,NaN,2023-02-23 00:00:00,NaN


In [44]:
analyst

,title,link,description,pubDate,location
357,Accounts Payable Analyst,https://cryptojobslist.com/jobs/accounts-payable-analyst-parity-technologies-london-england-united-kingdom,Tags Cryptocurrency Jobs Web3 Analyst Jobs Web3 Operations Jobs Web3 Finance Jobs Blockchain Accounting Jobs Blockchain Full Time Jobs Accou...,2023-02-17,London England United Kingdom
261,Binance Accelerator Programme Data analyst intern,https://cryptojobslist.com/jobs/binance-accelerator-programme-data-analyst-intern-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Blockchain Internship Jobs Web3 Analyst Jobs Cryptocurrency Engineering Jobs Blockchain Exc...,2023-02-23,Singapore
1168,Technical Analyst,https://remoteOK.com/jobs/201886,1kx is an earlystage fund that helps founders launch and bootstrap token networks We primarily lead Seed and Series A investment rounds with a foc...,NaT,Remote Worldwide
1202,Validator Research Analyst,https://remoteOK.com/jobs/200374,As stewards of the Polkadot and Substrate ecosystem Parity is laying the foundation for a better web which respects the freedom and data of indivi...,NaT,Worldwide


## Sales - Worldwide

In [45]:
sales = worldwide[worldwide['title'].str.contains('sales', case=False) | worldwide['description'].str.contains('sales', case=False)]

In [46]:
sales.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_13165/3195154911.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  sales.describe()


,title,link,description,pubDate,location
count,10,10,10,7,10
unique,10,10,10,3,3
top,Aha Senior Product Marketing Manager,https://weworkremotely.com/remote-jobs/aha-senior-product-marketing-manager,Headquarters United States URL httpwwwahaio Aha helps companies build lovable software We provide the worlds 1 product development tools Aha Ro...,2023-03-06 00:00:00,NaN
freq,1,1,1,4,7
first,NaN,NaN,NaN,2023-02-16 00:00:00,NaN
last,NaN,NaN,NaN,2023-03-06 00:00:00,NaN


In [47]:
sales

,title,link,description,pubDate,location
1354,Aha Senior Product Marketing Manager,https://weworkremotely.com/remote-jobs/aha-senior-product-marketing-manager,Headquarters United States URL httpwwwahaio Aha helps companies build lovable software We provide the worlds 1 product development tools Aha Ro...,2023-02-16,NaN
1327,Coachingcom Customer Success Agent,https://weworkremotely.com/remote-jobs/coaching-com-customer-success-agent,Headquarters United States URL httpscoachingcom GENERAL INFORMATIONDepartment Customer SuccessReporting to Support Manager Start Date Immediate...,2023-03-01,NaN
1328,TimeZest Growth Marketing Manager,https://weworkremotely.com/remote-jobs/timezest-growth-marketing-manager,Headquarters Remote URL httpstimezestcom About TimeZest TimeZest is a B2B SaaS technology company that has reimagined how the IT Service indus...,2023-03-01,NaN
1122,Director of Customer Success at Apollo,https://nodesk.co/remote-jobs/apollo-director-of-customer-success/,Apollo is hiring a remote Director of Customer Success This is a fulltime position that can be done remotely anywhere in the United States Apollo ...,2023-03-06,NaN
1295,Toggl Sales Development Representative Americas,https://weworkremotely.com/remote-jobs/toggl-sales-development-representative-americas-3,Headquarters Tallin URL httpstogglcomjobs Job Description We are looking for proactive and selfdriven Sales Development Representatives to fu...,2023-03-06,NaN
1288,Aha Product Success Manager Product management experience required,https://weworkremotely.com/remote-jobs/aha-product-success-manager-product-management-experience-required-1,Headquarters Remote URL httpsahaio Customer Success at Aha Aha helps companies build lovable software We provide the worlds 1 product developme...,2023-03-06,NaN
1287,Aha Sr Product Success Manager Product management experience required,https://weworkremotely.com/remote-jobs/aha-sr-product-success-manager-product-management-experience-required-1,Headquarters Remote URL httpsahaio Aha helps companies build lovable software We provide the worlds 1 product development tools Aha Roadmaps ...,2023-03-06,NaN
1167,DevOps Engineer,https://remoteOK.com/jobs/201896,At Figment our mission is to create an Internet that is truly decentralized where users can freely interact share collaborate and exchange goods a...,NaT,Worldwide
1192,Head of Quality Assurance,https://remoteOK.com/jobs/200396,Accelerating the Web3 transformation journeySettleMint is here to make blockchain application development easy for developers by providing the bes...,NaT,Remote job
1212,Developer Advocate,https://remoteOK.com/jobs/199859,About Swirlds LabsSwirlds Labs was established with the mission to accelerate the future built on Hedera the most used greenest enterprisegrade p...,NaT,Worldwide


## Python - Worldwide

In [48]:
python = worldwide[worldwide['title'].str.contains('python', case=False) | worldwide['description'].str.contains('python', case=False)]

In [49]:
python.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_13165/349435123.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  python.describe()


,title,link,description,pubDate,location
count,15,15,15,7,15
unique,15,15,15,6,4
top,Sr DevOps Engineer New Zealand Fiji GMT12,https://cryptojobslist.com/jobs/sr-devops-engineer-new-zealand-fiji-gmt-12-polygon-auckland,Tags Cryptocurrency Jobs Cryptocurrency Remote Jobs Blockchain Devops Jobs Web3 Security Jobs Blockchain Developer Jobs Web3 Engineering Job...,2023-03-03 00:00:00,NaN
freq,1,1,1,2,8
first,NaN,NaN,NaN,2023-02-17 00:00:00,NaN
last,NaN,NaN,NaN,2023-03-03 00:00:00,NaN


In [50]:
python

,title,link,description,pubDate,location
356,Sr DevOps Engineer New Zealand Fiji GMT12,https://cryptojobslist.com/jobs/sr-devops-engineer-new-zealand-fiji-gmt-12-polygon-auckland,Tags Cryptocurrency Jobs Cryptocurrency Remote Jobs Blockchain Devops Jobs Web3 Security Jobs Blockchain Developer Jobs Web3 Engineering Job...,2023-02-17,Auckland
1352,YAGRO LTD Senior DevOps Engineer,https://weworkremotely.com/remote-jobs/yagro-ltd-senior-devops-engineer,Headquarters Cambridge URL httpswwwyagrocom The agricultural industry is undergoing a profound change Tightening regulatory requirements reduc...,2023-02-20,NaN
261,Binance Accelerator Programme Data analyst intern,https://cryptojobslist.com/jobs/binance-accelerator-programme-data-analyst-intern-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Blockchain Internship Jobs Web3 Analyst Jobs Cryptocurrency Engineering Jobs Blockchain Exc...,2023-02-23,Singapore
1332,QuickMail Technical Support QA Tester USD 4050k 4 daysweek,https://weworkremotely.com/remote-jobs/quickmail-technical-support-qa-tester-usd-40-50k-4-days-week,Headquarters Europe URL httpsquickmailio Highlights Join a meritocracy no politics needed nor welcomed Work 4 days a week have an extra day...,2023-02-28,NaN
1315,FileCloud Senior Software Engineer C,https://weworkremotely.com/remote-jobs/filecloud-senior-software-engineer-cpp,Headquarters Austin TX URL httpswwwfilecloudcom FileCloud is looking for a Senior Software Engineer C with a strong track record to join our dy...,2023-03-02,NaN
1312,Publitascom Operations Engineer SaaS Remote,https://weworkremotely.com/remote-jobs/publitas-com-operations-engineer-saas-remote-1,Headquarters Netherlands URL httpspublitascom Create customer value We believe that business growth starts with a fantastic product that peopl...,2023-03-03,NaN
1308,Ivy ML Engineer Intern,https://weworkremotely.com/remote-jobs/ivy-ml-engineer-intern-4,Headquarters London GB URL httpsletsunifyai At Ivy were on a mission to unify all Machine Learning ML frameworks making ML code cleaner more fl...,2023-03-03,NaN
912,Fulltime Senior Full Stack Developer at Baserow,https://www.fossjobs.net/job/11192/senior-full-stack-developer-at-baserow/,Location WorldwideURL httpsbaserowiojobsexperiencedfullstackdeveloperDescription Baserow is on a mission to help everyone build applications faste...,NaT,NaN
915,Fulltime Senior Software Engineer Abstract Wikipedia at Wikimedia Foundation,https://www.fossjobs.net/job/11188/senior-software-engineer-abstract-wikipedia-at-wikimedia-foundation/,Location WorldwideURL httpsgrnhseb639756b1usDescription Senior Software Engineer Abstract Wikipedia Summary The Wikimedia Foundation is looking fo...,NaT,NaN
1154,Remote Senior Python Developer Scopic,https://www.python.org/jobs/7093/,Tirane Tirane Albania Are you on the hunt for exciting new challenges that boost your professional growth If youre an innovator by nature and a Py...,NaT,NaN


## SQL - Worldwide

In [51]:
sql = worldwide[worldwide['title'].str.contains('sql', case=False) | worldwide['description'].str.contains('sql', case=False)]

In [52]:
sql.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_13165/1629035654.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  sql.describe()


,title,link,description,pubDate,location
count,11,11,11,6,11
unique,11,11,11,6,4
top,Sr DevOps Engineer New Zealand Fiji GMT12,https://cryptojobslist.com/jobs/sr-devops-engineer-new-zealand-fiji-gmt-12-polygon-auckland,Tags Cryptocurrency Jobs Cryptocurrency Remote Jobs Blockchain Devops Jobs Web3 Security Jobs Blockchain Developer Jobs Web3 Engineering Job...,2023-02-17 00:00:00,NaN
freq,1,1,1,1,7
first,NaN,NaN,NaN,2023-02-17 00:00:00,NaN
last,NaN,NaN,NaN,2023-03-06 00:00:00,NaN


In [53]:
sql

,title,link,description,pubDate,location
356,Sr DevOps Engineer New Zealand Fiji GMT12,https://cryptojobslist.com/jobs/sr-devops-engineer-new-zealand-fiji-gmt-12-polygon-auckland,Tags Cryptocurrency Jobs Cryptocurrency Remote Jobs Blockchain Devops Jobs Web3 Security Jobs Blockchain Developer Jobs Web3 Engineering Job...,2023-02-17,Auckland
1352,YAGRO LTD Senior DevOps Engineer,https://weworkremotely.com/remote-jobs/yagro-ltd-senior-devops-engineer,Headquarters Cambridge URL httpswwwyagrocom The agricultural industry is undergoing a profound change Tightening regulatory requirements reduc...,2023-02-20,NaN
261,Binance Accelerator Programme Data analyst intern,https://cryptojobslist.com/jobs/binance-accelerator-programme-data-analyst-intern-binance-singapore,Tags Cryptocurrency Jobs Blockchain Trading Jobs Blockchain Internship Jobs Web3 Analyst Jobs Cryptocurrency Engineering Jobs Blockchain Exc...,2023-02-23,Singapore
1337,Skedda Platform Engineer Europe,https://weworkremotely.com/remote-jobs/skedda-platform-engineer-europe,Headquarters Melbourne Austrailia URL httpsskeddacom FULLTIME 100 Remote Within Europe or Central European Time Zone CET Ideal start timeline...,2023-02-27,NaN
1312,Publitascom Operations Engineer SaaS Remote,https://weworkremotely.com/remote-jobs/publitas-com-operations-engineer-saas-remote-1,Headquarters Netherlands URL httpspublitascom Create customer value We believe that business growth starts with a fantastic product that peopl...,2023-03-03,NaN
1298,Paymentology Data Architect,https://weworkremotely.com/remote-jobs/paymentology-data-architect,Headquarters London UK URL httpspaymentologycom Paymentology is the first truly global issuerprocessor giving banks and fintechs the technology...,2023-03-06,NaN
911,Fulltime DevOps Engineer at Baserow,https://www.fossjobs.net/job/11193/devops-engineer-at-baserow/,Location WorldwideURL httpsbaserowiojobsdevopsengineerDescription Baserow is on a mission to help everyone build applications faster cheaper and e...,NaT,NaN
912,Fulltime Senior Full Stack Developer at Baserow,https://www.fossjobs.net/job/11192/senior-full-stack-developer-at-baserow/,Location WorldwideURL httpsbaserowiojobsexperiencedfullstackdeveloperDescription Baserow is on a mission to help everyone build applications faste...,NaT,NaN
915,Fulltime Senior Software Engineer Abstract Wikipedia at Wikimedia Foundation,https://www.fossjobs.net/job/11188/senior-software-engineer-abstract-wikipedia-at-wikimedia-foundation/,Location WorldwideURL httpsgrnhseb639756b1usDescription Senior Software Engineer Abstract Wikipedia Summary The Wikimedia Foundation is looking fo...,NaT,NaN
1209,Senior Data Engineer,https://remoteOK.com/jobs/200380,About the roleAt CoW Protocol we have many opportunities to work with data each and every dayWe are looking for a Senior Data Engineer to support ...,NaT,Worldwide
